In [0]:
%sql
-- Explore Bronze (Raw Data)
SELECT count(*) FROM delta.`/Volumes/workspace/default/storage/bronze/ticker_data_v2`
LIMIT 10;

-- Explore Silver (Calculated Beta)
SELECT s , count(*) FROM delta.`/Volumes/workspace/default/storage/silver/ticker_data_v5`
group by  s 
ORDER BY window.start DESC
LIMIT 10;

DELETE FROM delta.`/Volumes/workspace/default/storage/silver/ticker_data`;

REMOVE '/Volumes/workspace/default/storage/checkpoints/job_silver_ticker';



WITH LaggedData AS (
    SELECT 
        s,
        event_time,
        -- Calculate difference in seconds
        unix_timestamp(event_time) - unix_timestamp(LAG(event_time) OVER (PARTITION BY s ORDER BY event_time)) as gap_seconds
    FROM delta.`/Volumes/workspace/default/storage/silver/ticker_data_v5`
)
SELECT 
    s,
    COUNT(*) as total_records,
    -- The Median (50th percentile)
    percentile_approx(gap_seconds, 0.5) as median_gap_seconds,
    -- Keep Avg for comparison
    ROUND(AVG(gap_seconds), 2) as avg_gap_seconds,
    MIN(gap_seconds) as min_gap,
    MAX(gap_seconds) as max_gap
FROM LaggedData
WHERE gap_seconds IS NOT NULL
GROUP BY s
ORDER BY s;